# Conservatory Project 2: GDP by State and County Y2015 - Y2019

This project has as its main goal the visualization of GDP Data of all the Counties found in each State.

## Summary of Project Steps

### Step 1: Extract and Transform

The first step is the focus of this notebook, to extract the data and perform subsequent transformations including cleaning, aggregating and formatting.

To extract the GDP data, API request is made to the website of the US Bureau of Economic Activity (BEA) which houses data on varios economic metrics. The response is then parsed and transformed into a desireable format to be loaded in a database. The transformation involves an iterative process of data cleaning and munging as is manifested in the notebook.

A second set of data is also extracted from Data.Health.Gov which has geolocation points of all counties in all 51 States. These points are required to plot maps and other geolocation charts. The transformation follows a similar process done in the first dataset extraction.

The above is fully demonstrated in this notebook.

### Step 2: Load

The load is the final step in the ETL process. It is an integral part of the data acquisition and management pipeline. 

This project will use MongoDB as database for storage purposes. The end structure of the datasets was taken into consideration when deciding to employ this Data Base Management System. The data will be transformed into a JSON and then loaded into the database. Please see **load.py** for more details.

### Step 3: Visualization

The main objective and final step in this project. The visualization will be rendered on the browser employing JavaScript and associated libraries such as D3.js, Leaflet.js and Plotly.js.

The data for the plots is retrieved from the MongoDB database via a Flask powered API (see **app.py**). A dashboard is then created comprisng of several chart types showing GDP of Counties and States across the time period of 2015 - 2019. The charts will be interactive and be able to update with dropdown and click actions.

The dashboard charts are in **index.html** while the codes are in **app.js**.

### Import dependencies

In [1]:
import requests
import pandas as pd
import json
from config import API_KEY

## Main Data Extracts

### Counties GDP Retrieval

In [2]:
"""
# Query assembly
table_id = "CAGDP9" # Table ID to input in API Endpoint
base_url = f'https://apps.bea.gov/api/data/?UserID={API_KEY}\
Key&method=GetData&datasetname=Regional&TableName={table_id}&LineCode=2&GeoFips=COUNTY&ResultFormat=json' # API Endpoint
"""

'\n# Query assembly\ntable_id = "CAGDP9" # Table ID to input in API Endpoint\nbase_url = f\'https://apps.bea.gov/api/data/?UserID={API_KEY}Key&method=GetData&datasetname=Regional&TableName={table_id}&LineCode=2&GeoFips=COUNTY&ResultFormat=json\' # API Endpoint\n'

In [3]:
# response
# gdp_response_data = requests.get(base_url).json()

In [4]:
"""
# Extract first data points
gdp_df = {} 
for key, val in gdp_response_data.items():
    gdp_df['data'] = val['Results']['Data']
"""

"\n# Extract first data points\ngdp_df = {} \nfor key, val in gdp_response_data.items():\n    gdp_df['data'] = val['Results']['Data']\n"

In [5]:
"""
# Extract main data points
county = []
year = []
gdp = []
county_geo_id = []
for vals in gdp_df.values():
    for val in vals:
        county_geo_id.append(val['GeoFips'])
        county.append(val['GeoName'])
        year.append(val['TimePeriod'])
        gdp.append(val['DataValue'])

# create data frame of data
df_gdp = pd.DataFrame({'GEOID': county_geo_id,
                       'county': county,
                       'year': year,
                       'gdp': gdp  
})
"""

"\n# Extract main data points\ncounty = []\nyear = []\ngdp = []\ncounty_geo_id = []\nfor vals in gdp_df.values():\n    for val in vals:\n        county_geo_id.append(val['GeoFips'])\n        county.append(val['GeoName'])\n        year.append(val['TimePeriod'])\n        gdp.append(val['DataValue'])\n\n# create data frame of data\ndf_gdp = pd.DataFrame({'GEOID': county_geo_id,\n                       'county': county,\n                       'year': year,\n                       'gdp': gdp  \n})\n"

In [6]:
# export as csv
# df_gdp.to_csv('counties_gdp_main.csv', index = False)

### States GDP Data Retrieval

In [7]:
# Extract States GDP
"""
table_id = "SAGDP9N" # Table ID to input in API Endpoint
base_url = f'https://apps.bea.gov/api/data/?UserID={API_KEY}\
Key&method=GetData&datasetname=Regional&TableName={table_id}&LineCode=1&GeoFips=STATE&ResultFormat=json' # API Endpoint
"""

'\ntable_id = "SAGDP9N" # Table ID to input in API Endpoint\nbase_url = f\'https://apps.bea.gov/api/data/?UserID={API_KEY}Key&method=GetData&datasetname=Regional&TableName={table_id}&LineCode=1&GeoFips=STATE&ResultFormat=json\' # API Endpoint\n'

In [8]:
# response states gdp
# gdp_response_data = requests.get(base_url).json()

In [9]:
# Extract main data points
"""
gdp_df = {} 
for key, val in gdp_response_data.items():
    gdp_df['data'] = val['Results']['Data']
    
geo_name = []
year = []
gdp = []
state_geo_id = []
for vals in gdp_df.values():
    for val in vals:
        state_geo_id.append(val['GeoFips'])
        geo_name.append(val['GeoName'])
        year.append(val['TimePeriod'])
        gdp.append(val['DataValue'])

df_gdp = pd.DataFrame({'state_id': state_geo_id,
                       'state_name': geo_name,
                       'year': year,
                       'gdp': gdp  
})

df_gdp.tail()
"""

"\ngdp_df = {} \nfor key, val in gdp_response_data.items():\n    gdp_df['data'] = val['Results']['Data']\n    \ngeo_name = []\nyear = []\ngdp = []\nstate_geo_id = []\nfor vals in gdp_df.values():\n    for val in vals:\n        state_geo_id.append(val['GeoFips'])\n        geo_name.append(val['GeoName'])\n        year.append(val['TimePeriod'])\n        gdp.append(val['DataValue'])\n\ndf_gdp = pd.DataFrame({'state_id': state_geo_id,\n                       'state_name': geo_name,\n                       'year': year,\n                       'gdp': gdp  \n})\n\ndf_gdp.tail()\n"

In [10]:
# Export df_gdp as csv
# df_gdp.to_csv('states_gdp_main.csv', index = False)

### Counties GeoFips Retrieval

In [11]:
# Query URL for API call to opendatasoft.com
# query_string = 'https://public.opendatasoft.com/api/records/1.0/search/?dataset=us-county-boundaries&q=&rows=3233&start=0&facet=statefp&facet=countyfp&facet=name&facet=stusab&facet=state_name&facet=GeoPoint&facet=GEOID'

In [12]:
# response
geoFips = requests.get(query_string).json()

In [13]:
# geoFips

In [14]:
# Extract main data points
"""
req_data = []
for key, val in geoFips.items():
    req_data.append(val)
    
df_fields = req_data[2]
"""

'\nreq_data = []\nfor key, val in geoFips.items():\n    req_data.append(val)\n    \ndf_fields = req_data[2]\n'

In [15]:
# Get plotting data
"""
geo_id = []
county = []
state = []
state_name = []
geo_point = []

for val in df_fields:
        
    geo_id.append(val['fields']['geoid'])
    county.append(val['fields']['name'])
    state.append(val['fields']['stusab'])
    state_name.append(val['fields']['state_name'])
    geo_point.append(val['fields']['geo_point_2d'])
"""

"\ngeo_id = []\ncounty = []\nstate = []\nstate_name = []\ngeo_point = []\n\nfor val in df_fields:\n        \n    geo_id.append(val['fields']['geoid'])\n    county.append(val['fields']['name'])\n    state.append(val['fields']['stusab'])\n    state_name.append(val['fields']['state_name'])\n    geo_point.append(val['fields']['geo_point_2d'])\n"

In [16]:
"""
# create a dataframe with plot data
geofips_df = pd.DataFrame({'GEOID': geo_id,
                          'county': county,
                          'state': state,
                          'state_name': state_name,
                          'geo_point': geo_point})
# Export as csv
geofips_df.to_csv('geofips_main.csv', index = False)
"""

"\n# create a dataframe with plot data\ngeofips_df = pd.DataFrame({'GEOID': geo_id,\n                          'county': county,\n                          'state': state,\n                          'state_name': state_name,\n                          'geo_point': geo_point})\n# Export as csv\ngeofips_df.to_csv('geofips_main.csv', index = False)\n"

## Main Transform

### Main Counties Data Transform

In [17]:
# Import counties_gdp_main.csv
counties_df = pd.read_csv('counties_gdp_main.csv')
counties_df.tail()

,GEOID,county,year,gdp
15575,56045,"Weston, WY",2015,"268,914"
15576,56045,"Weston, WY",2016,"232,683"
15577,56045,"Weston, WY",2017,"233,153"
15578,56045,"Weston, WY",2018,"253,497"
15579,56045,"Weston, WY",2019,"270,215"


In [18]:
# Import counties geoFips
geoFips_df = pd.read_csv('geofips_main.csv')
geoFips_df.head()

,GEOID,county,state,state_name,geo_point
0,42053,Forest,PA,Pennsylvania,"[41.5130724363, -79.2360118434]"
1,37001,Alamance,NC,North Carolina,"[36.0437098766, -79.3994248175]"
2,17025,Clay,IL,Illinois,"[38.7541714951, -88.4902374225]"
3,29103,Knox,MO,Missouri,"[40.1282548929, -92.1480723868]"
4,18115,Ohio,IN,Indiana,"[38.9500945902, -84.9650431209]"


In [19]:
# merge data on GEOID
merged_county_geoFips = counties_df.merge(geoFips_df, how = 'left', left_on= 'GEOID', right_on = 'GEOID', suffixes=('_x', '_y')) 

In [20]:
# merged_county_geoFips.tail()

In [21]:
# Drop county_x
merged_county_geoFips_drop_x = merged_county_geoFips.drop('county_x', axis = 1)
# Rename county_y to county
merged_county_geoFips_drop_x_county_renamed = merged_county_geoFips_drop_x.rename(columns = {'county_y': 'county'})

In [22]:
# merged_county_geoFips_drop_x_county_renamed['state_name'].unique()

In [23]:
# Transform GDP
num_conv = [x.replace(',', '') for x in merged_county_geoFips_drop_x_county_renamed['gdp']] # remove the commas in gdp values
merged_county_geoFips_drop_x_county_renamed['gdp'] = [x for x in num_conv]

In [24]:
# merged_county_geoFips_drop_x_county_renamed.head()

In [25]:
# check for null values in county, state, state_name and geo_point
# merged_county_geoFips_drop_x_county_renamed.info()

In [26]:
# Null values
null_df_counties = merged_county_geoFips_drop_x_county_renamed.loc[merged_county_geoFips_drop_x_county_renamed['county'].isna()]

In [27]:
# null_df_counties

In [28]:
# Drop Null rows
merged_county_geoFips_drop_x_county_renamed.dropna(inplace = True)

In [29]:
merged_county_geoFips_drop_x_county_renamed.reset_index(inplace = True, drop = True) # Reset index

In [30]:
merged_county_geoFips_drop_x_county_renamed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15445 entries, 0 to 15444
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   GEOID       15445 non-null  int64 
 1   year        15445 non-null  int64 
 2   gdp         15445 non-null  object
 3   county      15445 non-null  object
 4   state       15445 non-null  object
 5   state_name  15445 non-null  object
 6   geo_point   15445 non-null  object
dtypes: int64(2), object(5)
memory usage: 844.8+ KB


In [31]:
# Check for duplicates
# duplcates_merged_county_geoFips_drop_x_county_renamed = merged_county_geoFips_drop_x_county_renamed.duplicated()
# duplcates_merged_county_geoFips_drop_x_county_renamed.unique()

In [32]:
# Convert gdp to numeric for ranking
merged_county_geoFips_drop_x_county_renamed['gdp'] = pd.to_numeric(merged_county_geoFips_drop_x_county_renamed['gdp'], errors = 'coerce')

In [33]:
# Rank counties at state level
merged_county_geoFips_drop_x_county_renamed['state_rank'] = merged_county_geoFips_drop_x_county_renamed.groupby(['state', 'state_name', 'year'])['gdp'].rank("dense", ascending=False)

In [34]:
# Rank Counties at National Level
merged_county_geoFips_drop_x_county_renamed["national_rank"] = merged_county_geoFips_drop_x_county_renamed.groupby( 'year')['gdp'].rank("dense", ascending=False)

In [35]:
merged_county_geoFips_drop_x_county_renamed.tail()

,GEOID,year,gdp,county,state,state_name,geo_point,state_rank,national_rank
15440,56045,2015,268914,Weston,WY,Wyoming,"[43.8404850615, -104.567830048]",21.0,2400.0
15441,56045,2016,232683,Weston,WY,Wyoming,"[43.8404850615, -104.567830048]",22.0,2503.0
15442,56045,2017,233153,Weston,WY,Wyoming,"[43.8404850615, -104.567830048]",21.0,2497.0
15443,56045,2018,253497,Weston,WY,Wyoming,"[43.8404850615, -104.567830048]",20.0,2464.0
15444,56045,2019,270215,Weston,WY,Wyoming,"[43.8404850615, -104.567830048]",20.0,2449.0


In [36]:
# Convert state_rank and national_rank to int
merged_county_geoFips_drop_x_county_renamed[['state_rank', 'national_rank']] = merged_county_geoFips_drop_x_county_renamed[['state_rank', 'national_rank']].astype(int)

In [53]:
merged_county_geoFips_drop_x_county_renamed.head()

,GEOID,year,gdp,county,state,state_name,geo_point,state_rank,national_rank
0,1001,2015,1290521,Autauga,AL,Alabama,"[32.5349201866, -86.642749235]",24,1150
1,1001,2016,1354797,Autauga,AL,Alabama,"[32.5349201866, -86.642749235]",23,1104
2,1001,2017,1277080,Autauga,AL,Alabama,"[32.5349201866, -86.642749235]",24,1168
3,1001,2018,1324335,Autauga,AL,Alabama,"[32.5349201866, -86.642749235]",24,1161
4,1001,2019,1294907,Autauga,AL,Alabama,"[32.5349201866, -86.642749235]",24,1194


In [70]:
test_group = merged_county_geoFips_drop_x_county_renamed.groupby(['state_name', 'year'])['gdp'].max().to_frame(name = 'gdp').reset_index()


In [73]:
# test_group_df = pd.DataFrame(test_group)
test_group_df_sort = test_group.sort_values(by = ['gdp'], ascending = False)

In [79]:
test_group_df_sort.head(70)

,state_name,year,gdp
24,California,2019,661697170
23,California,2018,644506153
22,California,2017,633566149
21,California,2016,609904089
164,New York,2019,606498241
...,...,...,...
111,Michigan,2016,87172527
140,Nevada,2015,86140743
110,Michigan,2015,85474426
166,North Carolina,2016,84792210


In [38]:
# Export as json mainCountiesGeoFips.json
# merged_county_geoFips_drop_x_county_renamed.to_json('mainCountiesGeoFips.json', orient="records")

### Main States Data Transform

In [39]:
# Import states_gdp_main.csv
states_gdp_geoFips_df = pd.read_csv('states_gdp_main.csv')
states_gdp_geoFips_df.tail()

,state_id,state_name,year,gdp
295,98000,Far West,2015,"3,341,931.9"
296,98000,Far West,2016,"3,455,461.8"
297,98000,Far West,2017,"3,605,311.7"
298,98000,Far West,2018,"3,738,194.2"
299,98000,Far West,2019,"3,864,650.3"


In [40]:
# Transform GDP
states_num_conv = [x.replace(',', '') for x in states_gdp_geoFips_df['gdp']] # remove the commas in gdp values
states_gdp_geoFips_df['gdp'] = [x for x in states_num_conv]

In [41]:
# Format gdp and convert to numeric
states_gdp_geoFips_df['gdp'] = pd.to_numeric(states_gdp_geoFips_df['gdp'], errors = 'coerce')

In [42]:
# Check for all non states
states_gdp_geoFips_df['state_name'].unique()

array(['United States', 'Alabama', 'Alaska', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'New England', 'Mideast',
       'Great Lakes', 'Plains', 'Southeast', 'Southwest',
       'Rocky Mountain', 'Far West'], dtype=object)

In [43]:
# Extract US Totals
us_totals_df = states_gdp_geoFips_df.loc[states_gdp_geoFips_df['state_name'] == 'United States']
us_totals_df

,state_id,state_name,year,gdp
0,0,United States,2015,17432170.0
1,0,United States,2016,17730509.0
2,0,United States,2017,18144105.0
3,0,United States,2018,18687786.0
4,0,United States,2019,19091662.0


In [44]:
# Convert to usTotals.json
# us_totals_df.to_json('usTotals.json', orient="records")

In [45]:
# Get non states
non_states_df = states_gdp_geoFips_df.loc[(states_gdp_geoFips_df['state_name'] == 'United States') | (states_gdp_geoFips_df['state_name'] == 'Mideast')\
                                         | (states_gdp_geoFips_df['state_name'] == 'Great Lakes') | (states_gdp_geoFips_df['state_name'] == 'Plains')\
                                         | (states_gdp_geoFips_df['state_name'] == 'Southeast') | (states_gdp_geoFips_df['state_name'] == 'Southwest')\
                                         | (states_gdp_geoFips_df['state_name'] == 'Rocky Mountain') | (states_gdp_geoFips_df['state_name'] == 'Far West')\
                                         | (states_gdp_geoFips_df['state_name'] == 'New England')]

In [46]:
# Extract non states and us values
states_gdp_geoFips_real_df = states_gdp_geoFips_df[~states_gdp_geoFips_df.isin(non_states_df)].dropna()

In [47]:
states_gdp_geoFips_real_df_reset_index = states_gdp_geoFips_real_df.reset_index(drop = True)

In [48]:
states_gdp_geoFips_real_df_reset_index['rank'] = states_gdp_geoFips_real_df_reset_index.groupby('year')['gdp'].rank("dense", ascending=False)

In [49]:
# convert state_id, year and gdp to int
states_gdp_geoFips_real_df_reset_index[['state_id', 'year', 'gdp', 'rank']] = states_gdp_geoFips_real_df_reset_index[['state_id', 'year', 'gdp', 'rank']].astype(int)

In [50]:
# states_gdp_geoFips_real_df_reset_index.info()

In [51]:
# convert to mainStatesGdp.json
# states_gdp_geoFips_real_df_reset_index.to_json('mainStatesGdp.json', orient="records")